In [ ]:
pip install git+https://github.com/openai/CLIP.git

In [ ]:
! pip  install clip

In [1]:
import os
import clip
import torch
from PIL import Image

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

In [2]:
# Path to the folder containing images
folder_path = "processed_images"
files = os.listdir(folder_path)

In [3]:
batch_size = 1000
total_files = len(files)
n_batch_iter = int(total_files/batch_size)
desired_feature_dimension = 512

# Initialize a tensor to accumulate image features
all_image_features = torch.empty(0, desired_feature_dimension).to(device)  # Assuming desired_feature_dimension is known
for i in range(n_batch_iter):
    print(f"Batch id: {i}")
    batch_sample = files[i*batch_size:(i*batch_size + batch_size)]
    image_list = []    
    for file in batch_sample:
        image_list.append(Image.open(folder_path + '/' + file))
    batch_image_input = torch.stack([preprocess(image) for image in image_list]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(batch_image_input)    
    image_features /= image_features.norm(dim=-1, keepdim=True)
    print(image_features.shape)
    all_image_features = torch.cat((all_image_features, image_features), dim=0)


Batch id: 0
torch.Size([1000, 512])
Batch id: 1
torch.Size([1000, 512])
Batch id: 2
torch.Size([1000, 512])
Batch id: 3
torch.Size([1000, 512])
Batch id: 4
torch.Size([1000, 512])
Batch id: 5
torch.Size([1000, 512])
Batch id: 6
torch.Size([1000, 512])
Batch id: 7
torch.Size([1000, 512])
Batch id: 8
torch.Size([1000, 512])
Batch id: 9
torch.Size([1000, 512])


In [4]:
import pickle
# Save the tensor to a pickle file
with open('processed_image_features.pkl', 'wb') as f:
    pickle.dump({"image_feature":all_image_features, "files":files}, f)

# Load the pickle file
with open('processed_image_features.pkl', 'rb') as f:
    data = pickle.load(f)


In [5]:
data['image_feature'].shape

torch.Size([10000, 512])

In [17]:
image_list[0].size

(416, 416)

In [10]:
batch_image_input.shape

torch.Size([1000, 3, 224, 224])